In [ ]:
suppressMessages(library(tidyr))
suppressMessages(library(CausalImpact))


In [ ]:
suppressMessages(library(pscl))
suppressMessages(library(ggplot2))
suppressMessages(require(pscl))
suppressMessages(require(boot))

In [ ]:
suppressMessages(library(repr))

In [ ]:
suppressMessages(library(dplyr))

In [ ]:
suppressMessages(library(reshape))

In [ ]:
save_tsv <- function(data, name, start){
    tmp <- data$series
    time.points <- seq.Date(as.Date(start), by = 1, length.out = length(tmp$response))
    tmp <- zoo(tmp, time.points)
    write.table(tmp, name, sep="\t")
}

In [95]:
get_data <- function(start, end, country, grp) {
    qry <- paste(sprintf("from usersession %s %s where ", start, end),
                 sprintf("bot=0 country='%s' landing_page_type='acme_title_salaries' session_from='googleorg' group by time(1d) ", country),
                 sprintf("select grp='%s', grp='acme_seo1_tst0', grp='acme_seo1_tst6', ", grp),
                 "grp='acme_seo1_tst9' ", sep='')
   if(FALSE) {
   qry <- paste(sprintf("from usersession %s %s where ", start, end),
                 sprintf("bot=0 country='%s' landing_page_type='acme_title_salaries' session_from='googleorg' group by time(1d) ", country),
                 sprintf("select grp='%s', ", grp),
                 "grp='acme_seo1_tst0' ", sep='')
    }
    print(qry)
    data <- generic_iql(qry, suppress=TRUE)
    rownames(data) <- data$time
    names(data) <- c("time", "test", "control0", "control1", "control2")
    keep <- c("test", "control0", "control1", "control2")
    #names(data) <- c("time", "test", "control")
    #keep <- c("test", "control")
    time.points <- seq.Date(as.Date(start), by = 1, length.out = length(data$time))
    data <- zoo(data[keep], time.points)
    return (data)
}

In [96]:
predict <- function(start, end, intervention, country, grp) {
    data <- get_data(start, end, country, grp)
    pre.period <- as.Date(c(start, as.character(as.Date(intervention)-1)))
    post.period <- as.Date(c(intervention, end))
    impact <- CausalImpact(data, pre.period, post.period, alpha = 0.1)
    return (impact)
}

In [97]:
results <- function(data) {
    cat('posterior: ',format(round(100*(1 - data$summary["Cumulative","p"]), 2), nsmall=2), '\n')
    cat('cumulative effect: ', format(round(100*data$summary["Cumulative","RelEffect"], 2), nsmall=2), '\n')
    cat('cumulative effect lower: ', format(round(100*data$summary["Cumulative","RelEffect.lower"], 2), nsmall=2), '\n')
    cat('cumulative effect upper: ', format(round(100*data$summary["Cumulative","RelEffect.upper"], 2), nsmall=2), '\n')
}

In [98]:
save_result <- function(data, end, filename='in.csv') {
    data = c(end, 
             data$summary["Cumulative","RelEffect"],
             data$summary["Cumulative","RelEffect.lower"],
             data$summary["Cumulative","RelEffect.upper"], 
             1 - data$summary["Cumulative","p"])
    write.csv(data, filename)
}

In [99]:
display <- function(data, intervention, end) {
    results(data)
}

In [104]:
run <- function(start, end, intervention, country, grp) {
    seo <- predict(start, end, intervention, country, grp) 
    display(seo, intervention, end)
    options(repr.plot.width=8, repr.plot.height=5)
    seo$summary
    save_tsv(seo, 'test.tsv', start)
    plot(seo, c("original", "pointwise", "cumulative"))
    #plot(seo$model$bsts.model, "coefficients")
}

In [31]:
generic_iql <- function(query, suppress=FALSE) {
    if (suppress) {
        data <- suppressMessages(iql(query))
    } else {
        data <- iql(query)
    }
    return (data)
}

In [9]:
suppressMessages(library(zoo))
library(shiny)
#suppressMessages(library(iqlr))
suppressMessages(library(CausalImpact))
suppressMessages(library("dplyr"))

clean_iql_date <- function(data) {
    return (substr(data, start = 2, stop = 11))
}

show_results <- function(data) {
    paste0(
    paste('posterior: ',format(round(100*(1 - data$summary["Cumulative","p"]), 2), nsmall=2), '%\n'),
    paste('cumulative effect: ', format(round(100*data$summary["Cumulative","RelEffect"], 2), nsmall=2), '%\n'),
    paste('cumulative effect lower: ', format(round(100*data$summary["Cumulative","RelEffect.lower"], 2), nsmall=2), '%\n'),
    paste('cumulative effect upper: ', format(round(100*data$summary["Cumulative","RelEffect.upper"], 2), nsmall=2), '%\n')
    )
}

generic_iql <- function(query, suppress=FALSE) {
    if (suppress) {
        data <- suppressMessages(iql(query))
    } else {
        data <- iql(query)
    }
    return (data)
}

# Define UI for data upload app ----
ui <- fluidPage(

  # App title ----
  titlePanel("Upload File"),

  # Sidebar layout with input and output definitions ----
  sidebarLayout(

    # Sidebar panel for inputs ----
    sidebarPanel(

      # Input: Select a file ----
      fileInput("file1", "Choose CSV File",
                multiple = TRUE,
                accept = c("text/csv",
                         "text/comma-separated-values,text/plain",
                         ".csv")),

      # Horizontal line ----
      tags$hr(),

      # Input: Checkbox if file has header ----
      checkboxInput("header", "File has header?", TRUE),

      # Input: Select separator ----
      radioButtons("sep", "Separator",
                   choices = c(Tab = "\t",
                               Comma = ","),
                   selected = "\t"),

      # Horizontal line ----
      tags$hr(),
      dateInput("intervention", "Intervention:", value = "2018-07-01", format = "yyyy-mm-dd"),
      numericInput("alpha", "alpha:", 0.10, min = 0.1, max = 0.15, step=0.05)

    ),

    # Main panel for displaying outputs ----
    mainPanel(

    
      # Output: Data file ----
      plotOutput("plot1"),
      verbatimTextOutput("info"),
      tableOutput("final")

    )

  )
)


# Define server logic to read selected file ----
server <- function(input, output) {
  
  pull_data <- reactive({    
      req(input$file1)  
      df <- read.csv(input$file1$datapath,
             header = input$header,
             sep = input$sep,
             quote = "")
      df[c(1)] <- lapply(df[c(1)], clean_iql_date)
      n_columns = ncol(df)
      cols <- c("date", "test")
      for (i in 1:(n_columns - 2)) {
        cols <- c(cols, paste0("control", i))
      }
      names(df) <- cols
      rownames(df) <- df$date
      time.points <- seq.Date(as.Date(input$intervention), by = 1, length.out = length(df$date))
      intervention <- as.character(input$intervention)
      start <- df$date[1]
      end <- df$date[length(df$date)]
      for (i in 1:(n_columns - 1)) {
          j = i + 1
          df[, c(j)] =  as.numeric(df[, c(j)])  
      }
      df <- zoo(df, time.points)
      return(df)
  })

        

  ci <- reactive({      
    req(input$file1)  
    data <- read.csv(input$file1$datapath,
             header = input$header,
             sep = input$sep,
             quote = "")
    data[c(1)] <- lapply(data[c(1)], clean_iql_date)
    start = as.character(data[c(1)][,1][1])
    intervention = as.character(input$intervention)
    end = as.character(data[c(1)][,1][nrow(data)])
    pre.period <- as.Date(c(start, as.character(as.Date(intervention)-1)))
    post.period <- as.Date(c(intervention, end))
      
    keep <- c()
    for (i in 2:(ncol(data))) {
        keep <- c(keep, i)
    }

    time.points <- seq.Date(as.Date(start), by = 1, length.out = nrow(data))
    data <- zoo(data[keep], time.points)

      
    pre.period <- as.Date(c(start,  as.character(as.Date(intervention)-1)))
    post.period <- as.Date(c(intervention, end))
    cx <- CausalImpact(data, pre.period, post.period, alpha=input$alpha)

    return(cx)

  })
    
    
  output$final <- renderTable({

    # input$file1 will be NULL initially. After the user selects
    # and uploads a file, head of that data file by default,
    # or all rows if selected, will be shown.

    df <- ci()
    return(df$series)


  })
    
  output$plot1 <-renderPlot({
    plot(ci())
  })
    
  output$info <- renderText({
     data <- ci()
     show_results(data)
  })


}

# Create Shiny app ----
shinyApp(ui, server, options=list(port = 8080))


Listening on http://127.0.0.1:8080


In [287]:
sample <- function() {
    x <- read.csv('ci_test.tsv',
             header = FALSE,
             sep = '\t',
             quote = "")
    x[c(1)] <- lapply(x[c(1)], clean_iql_date)
    s <- as.character(x[c(1)][,1][1])
    inter <- as.character("2018-07-01")
    e <- as.character(x[c(1)][,1][nrow(x)])

    keep <- c()
    for (i in 2:(ncol(x))) {
        keep <- c(keep, i)
    }
    time.points <- seq.Date(as.Date(s), by = 1, length.out = nrow(x))
    data <- zoo(x[keep], time.points)
    
    pre.period <- as.Date(c(s,  as.character(as.Date(inter)-1)))
    post.period <- as.Date(c(inter, e))
    cx <- CausalImpact(data, pre.period, post.period)
    return(cx)
    }

In [288]:
sample()

Posterior inference {CausalImpact}

                         Average          Cumulative      
Actual                   45842            458417          
Prediction (s.d.)        5e+04 (935)      5e+05 (9351)    
95% CI                   [47838, 51444]   [478383, 514437]
                                                          
Absolute effect (s.d.)   -3787 (935)      -37873 (9351)   
95% CI                   [-5602, -1997]   [-56020, -19966]
                                                          
Relative effect (s.d.)   -7.6% (1.9%)     -7.6% (1.9%)    
95% CI                   [-11%, -4%]      [-11%, -4%]     

Posterior tail-area probability p:   0.00202
Posterior prob. of a causal effect:  99.79818%

For more details, type: summary(impact, "report")


In [ ]:
#https://support.rstudio.com/hc/en-us/articles/213733868-Running-Shiny-Server-with-a-Proxy